In [33]:
import pandas as pd
from google.colab import drive
from itertools import combinations

drive.mount('/content/drive')

path = '/content/drive/My Drive/NBA_Stats.xlsx'

# Read the Excel file into a DataFrame.
df = pd.read_excel(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
current_season = []

for i, player in df.iterrows():
  if(player["SeasonStartYear"] == 2018 and player["SeasonType"] == "Regular Season"):
      current_season.append(player)

df_2223 = pd.DataFrame(current_season)

In [ ]:
df_2223

<br>

### Find pairs of players whose average PTS/gm stats are similar based on a defined margin.

Generate list of combinations of players, without duplicates, to create subset of pairs whose average assists per game are within a defined margin of each other.

In [43]:
player_pairs = list(combinations(df_2223.itertuples(index=False), 2))
df_pts_pairs = pd.DataFrame()

margin = 0.75

for pair in player_pairs:
  playerAstatpg = pair[0].PointsScored / pair[0].GamesPlayed
  playerBstatpg = pair[1].PointsScored / pair[1].GamesPlayed
  if(pair[0].GamesPlayed > 10 and pair[1].GamesPlayed > 10):
    answer = pair[0].Player if playerAstatpg > playerBstatpg else pair[1].Player
    diff = abs(playerBstatpg - playerAstatpg)
    if diff < margin and diff >= 0.25 and playerAstatpg > 7:
      df_pts_pairs = pd.concat([df_pts_pairs, pd.DataFrame.from_records([{
      "PlayerAId": pair[0].PlayerID,
      "PlayerAName": pair[0].Player,
      "PlayerAStatperGame": playerAstatpg,
      "PlayerATeam": pair[0].Team,
      "PlayerBId": pair[1].PlayerID,
      "PlayerBName": pair[1].Player,
      "PlayerBStatperGame": playerBstatpg,
      "PlayerBTeam": pair[1].Team,
      "SeasonStart": pair[1].SeasonStartYear,
      "Answer": answer,
      "StatCategory": "PTS",
      "Interval": diff

      }])])

print(len(df_pts_pairs))

2588


In [ ]:
df_pts_pairs

### Find pairs of players whose average AST/gm stats are similar based on a defined margin.

In [44]:
player_pairs = list(combinations(df_2223.itertuples(index=False), 2))
df_ast_pairs = pd.DataFrame()

margin = 2.5

for pair in player_pairs:
  playerAstatpg = pair[0].Assists / pair[0].GamesPlayed
  playerBstatpg = pair[1].Assists / pair[1].GamesPlayed
  if(pair[0].GamesPlayed > 10 and pair[1].GamesPlayed > 10):
    answer = pair[0].Player if playerAstatpg > playerBstatpg else pair[1].Player
    diff = abs(playerBstatpg - playerAstatpg)
    if diff < margin and diff >= 0.25 and playerAstatpg > 4:
      df_ast_pairs = pd.concat([df_ast_pairs, pd.DataFrame.from_records([{
      "PlayerAId": pair[0].PlayerID,
      "PlayerAName": pair[0].Player,
      "PlayerAStatperGame": playerAstatpg,
      "PlayerATeam": pair[0].Team,
      "PlayerBId": pair[1].PlayerID,
      "PlayerBName": pair[1].Player,
      "PlayerBStatperGame": playerBstatpg,
      "PlayerBTeam": pair[1].Team,
      "SeasonStart": pair[1].SeasonStartYear,
      "Answer": answer,
      "StatCategory": "AST",
      "Interval": diff

      }])])

print(len(df_ast_pairs))
# df_ast_pairs.to_csv("AST_2022.csv")

3508


In [ ]:
df_ast_pairs

### Find pairs of players whose average REB/gm stats are similar based on a defined margin.

In [53]:
player_pairs = list(combinations(df_2223.itertuples(index=False), 2))
df_reb_pairs = pd.DataFrame()

margin = 1.4

for pair in player_pairs:
  playerAstatpg = pair[0].Rebounds / pair[0].GamesPlayed
  playerBstatpg = pair[1].Rebounds / pair[1].GamesPlayed
  if(pair[0].GamesPlayed > 10 and pair[1].GamesPlayed > 10):
    answer = pair[0].Player if playerAstatpg > playerBstatpg else pair[1].Player
    diff = abs(playerBstatpg - playerAstatpg)
    if diff < margin and diff > 0.2 and playerAstatpg > 5.2:
      df_reb_pairs = pd.concat([df_reb_pairs, pd.DataFrame.from_records([{
      "PlayerAId": pair[0].PlayerID,
      "PlayerAName": pair[0].Player,
      "PlayerAStatperGame": playerAstatpg,
      "PlayerATeam": pair[0].Team,
      "PlayerBId": pair[1].PlayerID,
      "PlayerBName": pair[1].Player,
      "PlayerBStatperGame": playerBstatpg,
      "PlayerBTeam": pair[1].Team,
      "SeasonStart": pair[1].SeasonStartYear,
      "Answer": answer,
      "StatCategory": "REB",
      "Interval": diff

      }])])

print(len(df_reb_pairs))
# df_reb_pairs.to_csv("AST_2022.csv)

2798


In [ ]:
df_reb_pairs

### Firestore update for all stats

In [55]:
import os
from google.cloud import firestore
from google.colab import auth

auth.authenticate_user()

In [56]:
# Combine all resulting dfs
df_pra = pd.DataFrame()
df_pra = pd.concat([df_pts_pairs, df_ast_pairs, df_reb_pairs], ignore_index=True)

In [57]:
import random

queued_questions = 0
included_players = []
while queued_questions < 10:
  index = random.randint(0,len(df_pra))
  single_doc = df_pra.iloc[index].to_dict()
  if(single_doc["PlayerAName"] in included_players or single_doc["PlayerBName"] in included_players):
    pass
  else:
    included_players.append(single_doc["PlayerAName"])
    included_players.append(single_doc["PlayerBName"])
    project_id = "vue-nba-guess"
    db = firestore.Client(project=project_id)
    doc_ref = db.collection("daily-challenge").document("2023-07-27").collection("questions").document(str(queued_questions))
    doc_ref.set(single_doc)
    queued_questions = queued_questions + 1


### Firestore update for only AST

In [ ]:
import os
from google.cloud import firestore
from google.colab import auth

auth.authenticate_user()

In [ ]:
import random

queued_questions = 0
included_players = []
while queued_questions < 10:
  index = random.randint(0,len(df_ast_pairs))
  single_doc = df_ast_pairs.iloc[index].to_dict()
  if(single_doc["PlayerAName"] in included_players or single_doc["PlayerBName"] in included_players):
    pass
  else:
    included_players.append(single_doc["PlayerAName"])
    included_players.append(single_doc["PlayerBName"])
    project_id = "vue-nba-guess"
    db = firestore.Client(project=project_id)
    doc_ref = db.collection("daily-challenge").document("2023-07-27").collection("questions").document(str(queued_questions))
    # doc_ref.set(single_doc)
    queued_questions = queued_questions + 1


<br>